# Encontro 13: Medidas de Centralidade

Importando a biblioteca:

In [61]:
import sys
sys.path.append('..')

from random import choice
from itertools import permutations

import pandas as pd
import networkx as nx

import socnet as sn

from scipy import stats

Configurando a biblioteca:

In [14]:
sn.node_size = 10
sn.node_color = (255, 255, 255)

sn.edge_width = 1
sn.edge_color = (192, 192, 192)

sn.node_label_position = 'top center'

Carregando rede de casamentos entre famílias de Florença durante a Renascença.

J. F. Padgett e C. K. Ansell. *Robust action and the rise of the Medici, 1400–1434.* American Journal of
Sociology 98, págs. 1259-1319, 1993.

In [15]:
g = sn.load_graph('Renaissance.gml', has_pos=True)

sn.show_graph(g, nlab=True)

Função que registra, em cada nó, seus sucessores em geodésicas de $s$ a $t$.

In [16]:
def set_geodesic_successors(g, s, t):
    for n in g.nodes:
        g.nodes[n]['geodesic_successors'] = set()

    for p in nx.all_shortest_paths(g, s, t):
        for i in range(len(p) - 1):
            g.nodes[p[i]]['geodesic_successors'].add(p[i + 1])

Funções que representam uma escolha aleatória de sucessor para diferentes tipos de trajetórias.

In [17]:
# Pense que o atributo 'passages' abaixo indica quantas
# vezes um fluxo já passou por um nó ou por uma aresta.

def random_geodesic_successor(g, n):
    return choice([m for m in g.nodes[n]['geodesic_successors']])

def random_path_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.nodes[m]['passages'] == 0])

def random_trail_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.edges[n, m]['passages'] == 0])

def random_walk_successor(g, n):
    return choice([m for m in g.neighbors(n)])

Função que faz uma simulação de fluxo de $s$ a $t$, que pode ou não ser bem-sucedida.

In [18]:
def simulate_single_flow(g, s, t, func, dup):
    # Inicializa o atributo 'passages' de cada nó.
    for n in g.nodes:
        g.nodes[n]['passages'] = 0
    g.nodes[s]['passages'] = 1

    # Inicializa o atributo 'passages' de cada aresta.
    for n, m in g.edges:
        g.edges[n, m]['passages'] = 0

    # Inicializa s como o único dono do insumo.
    for n in g.nodes:
        g.nodes[n]['owner'] = False
    g.nodes[s]['owner'] = True

    # Simula o fluxo, contando o número total de passos.
    steps = 0

    while True:
        # O conjunto reached representa todos os nós
        # que o fluxo consegue alcançar no passo atual.
        reached = set()

        # Verifica cada um dos donos atuais do insumo.
        owners = [n for n in g.nodes if g.nodes[n]['owner']]

        for n in owners:
            # Escolhe aleatoriamente um dos sucessores.
            try:
                if ( func == 'geo'):
                    m = random_geodesic_successor(g, n)
                elif (func == 'path'):
                    m = random_path_successor(g, n)
                elif (func == 'trail'):
                    m = random_trail_successor(g, n)
                else:
                     m = random_walk_successor(g, n)
            except IndexError:
                continue

            # Deixa de ser dono do insumo.
            if dup == 'dup':
                g.nodes[n]['owner'] = True
            else:
                g.nodes[n]['owner'] = False

            # Incrementa o atributo 'passages' do nó.
            g.nodes[m]['passages'] += 1

            # Incrementa o atributo 'passages' da aresta.
            g.edges[n, m]['passages'] += 1

            # Registra que consegue alcançar esse nó.
            reached.add(m)

        # Todo nó alcançado passa a ser dono do insumo.
        for n in reached:
            g.nodes[n]['owner'] = True

        # Isso conclui o passo atual da simulação.
        steps += 1

        # Se o passo alcançou t, chegamos ao fim da simulação.
        # Ela foi bem-sucedida: devolvemos o número de passos.
        if t in reached:
            return steps

        # Se o passo não alcançou ninguém, chegamos ao fim da
        # simulação. Ela não foi bem-sucedida: devolvemos -1.
        if not reached:
            return -1

Função que faz simulações de fluxo de $s$ a $t$ até uma ser bem-sucedida.

In [19]:
def simulate_successful_flow(g, s, t, func, dup):
    set_geodesic_successors(g, s, t)

    while True:
        steps = simulate_single_flow(g, s, t, func, dup)

        if steps != -1:
            return steps

Função que faz simulações de fluxo para todo $s$ e $t$ possíveis, e tira disso um *closeness simulado* e um *betweenness simulado*.

In [20]:
def simulate_all_flows(g, func, dup):
    for n in g.nodes:
        g.nodes[n]['closeness'] = 0
        g.nodes[n]['betweenness'] = 0

    for s, t in permutations(g.nodes, 2):
        steps = simulate_successful_flow(g, s, t, func, dup)

        g.nodes[s]['closeness'] += steps
        for n in g.nodes:
            if n != s and n != t:
                g.nodes[n]['betweenness'] += g.nodes[n]['passages']

    # Normalizações necessárias para comparar com os
    # resultados analíticos. Não é preciso entender.
    for n in g.nodes:
        g.nodes[n]['closeness'] = (g.number_of_nodes() - 1) / g.nodes[n]['closeness']
        g.nodes[n]['betweenness'] /= (g.number_of_nodes() - 1) * (g.number_of_nodes() - 2)

Média de *closeness simulado* e *betweenness simulado* para muitas repetições da simulação acima.

In [108]:
def main(tipo, dup):
    TIMES = 100
    
    #Dicionário com valores de centralidade (closeness e betweenness) para todas as famíilas (porém de apenas um trajetória e com ou sem duplicação)
    list_dict = {}

    for n in g.nodes:
#         g.nodes[n]['clos_list'] = []
#         g.nodes[n]['bet_list'] =[]
        
        clos_list = []
        bet_list = []
#         list_dict[g.nodes[n]['label']+'_clos_'+tipo+'_'+dup+'_list'] = g.nodes[n]['clos_list']
#         list_dict[g.nodes[n]['label']+'_bet_'+tipo+'_'+dup+'_list'] = g.nodes[n]['bet_list']
        list_dict[g.nodes[n]['label']+'_clos_'+tipo+'_'+dup+'_list'] = clos_list
        list_dict[g.nodes[n]['label']+'_bet_'+tipo+'_'+dup+'_list'] = bet_list
        print (g.nodes[n]['label']+'_bet_'+tipo+'_'+dup+'_list')

    for _ in range(TIMES):
        simulate_all_flows(g, tipo, dup)

        for n in g.nodes:
            list_dict[g.nodes[n]['label']+'_clos_'+tipo+'_'+dup+'_list'].append(g.nodes[n]['closeness'])
            list_dict[g.nodes[n]['label']+'_bet_'+tipo+'_'+dup+'_list'].append(g.nodes[n]['betweenness'])
#             clos_list.append(g.nodes[n]['closeness'])
#             bet_list.append(g.nodes[n]['betweenness'])

#             print(list_dict['medici_clos_geo_dup_list'])

    return (list_dict)

In [109]:
m = main('geo','notdup')

ginori_bet_geo_notdup_list
lambertes_bet_geo_notdup_list
albizzi_bet_geo_notdup_list
guadagni_bet_geo_notdup_list
pazzi_bet_geo_notdup_list
salviati_bet_geo_notdup_list
medici_bet_geo_notdup_list
tornabuon_bet_geo_notdup_list
bischeri_bet_geo_notdup_list
ridolfi_bet_geo_notdup_list
acciaiuol_bet_geo_notdup_list
strozzi_bet_geo_notdup_list
peruzzi_bet_geo_notdup_list
barbadori_bet_geo_notdup_list
castellan_bet_geo_notdup_list


Cálculo de *closeness* e *betweenness* a partir das funções prontas da NetworkX, para comparação.

In [57]:
# cc = nx.closeness_centrality(g)

# bc = nx.betweenness_centrality(g)

Construção de data frame só para comparar mais facilmente.

In [58]:
# pd.DataFrame({
#     'família': [g.nodes[n]['label'] for n in g.nodes],
#     'closeness simulado': [g.nodes[n]['mean_closeness'] for n in g.nodes],
#     'closeness analítico': [cc[n] for n in g.nodes],
#     'betweenness simulado': [g.nodes[n]['mean_betweenness'] for n in g.nodes],
#     'betweenness analítico': [bc[n] for n in g.nodes],
# })

E agora, vamos pensar um pouco...

* Onde você precisa mudar o código para usar uma *trajetória* que não seja a *geodésica*? (caminho, trilha, passeio)

* Onde você precisa mudar o código para usar uma *difusão* que não seja a *transferência*? (duplicação)

Considere então a seguinte **hipótese**:

>Quando consideramos outros tipos de trajetória e outros tipos de difusão, os nós com maior *closeness simulado* e *betweenness simulado* não são necessariamente os nós com maior *closeness* e *betweenness* segundo as fórmulas clássicas. (que correspondem ao uso de geodésica e transferência na simulação)

Queremos:

1. Operacionalização e teste dessas hipótese. (Objetivo 3)
2. Interpretação dos resultados na linguagem de Análise de Redes Sociais (Objetivo 4)

Um *feedback* da atividade sobre *coreness no Jazz* será dado em breve, para vocês terem uma melhor referência do item 2.

Montando dicionários com os valores de centralidade (closeness e betweenness) para cada amostra (todas as famílias sendo testadas com todos os tipos de trajetória ora tendo transferência, ora não)

In [99]:
all_dict = {}

all_dict['dup_geo_bet_clos_dict'] = main('geo','dup')
all_dict['dup_path_bet_clos_dict'] = main('path','dup')
all_dict['dup_trail_bet_clos_dict'] = main('trail','dup')
all_dict['dup_walk_bet_clos_dict'] = main('walk','dup')
all_dict['notdup_geo_bet_clos_dict'] = main('geo','notdup')
all_dict['notdup_path_bet_clos_dict']= main('path','notdup')
all_dict['notdup_trail_bet_clos_dict'] = main('trail','notdup')
all_dict['notdup_walk_bet_clos_dict'] = main('walk','notdup')


Analisando as listas dos valores de centralidade nos diferentes casos com os valores padrão (valores de centralidade utilizando geodésica com transferência)

In [113]:
fam_list = []
traject_list = ['geo', 'path', 'trail', 'walk']
dup_list = ['dup', 'notdup']
bet_clos_list = ['bet', 'clos']

# bench = all_dict['dup_geo_bet_clos_dict']['ginori_bet_geo_dup_list']

#Adicionando nomes das famílias na lista
for n in g.nodes:
    fam_list.append(g.nodes[n]['label'])

#Realizando teste t para as combinações
for fam in fam_list:
    for traject in traject_list:
        for dup in dup_list:
            for betclos in bet_clos_list:
                ttest = stats.ttest_ind(all_dict[''+dup+'_'+traject+'_bet_clos_dict'][''+fam+'_'+betclos+'_'+traject+'_'+dup+'_list'],all_dict['dup_geo_bet_clos_dict'][''+fam+'_'+betclos+'_geo_dup_list']) 
                print(ttest[1])

# print(stats.ttest_ind(all_dict['dup_geo_bet_clos_dict']['ginori_bet_geo_dup_list'],all_dict['dup_geo_bet_clos_dict']['medici_bet_geo_dup_list']))

nan
1.0
nan
1.0
3.390247899153548e-209
2.071568520722389e-163
nan
1.1679856857445387e-120
1.0232979076104645e-187
1.0387796829221932e-147
nan
1.2590706232913637e-117
5.023439399741181e-156
4.1831212838400326e-166
2.2650928990393244e-145
5.242286527972204e-253
nan
1.0
nan
1.0
3.3418042438108896e-194
4.114199264236295e-161
nan
9.967009253979772e-106
2.014565569696985e-200
2.528036114008436e-153
nan
2.445962130926418e-116
1.0551479089897812e-151
2.669084530828243e-165
1.329747691431473e-144
2.1945336203179653e-229
1.0
1.0
4.821299035086165e-188
1.0
2.8084722235689025e-29
1.954706985729343e-182
1.0003702767185109e-169
3.839974489409716e-134
1.796211339719951e-71
5.887737660490382e-187
3.062300192235681e-155
1.3395566716679802e-106
4.158738047272377e-153
4.030232147586517e-183
1.2989123218170107e-147
2.744920628728578e-283
1.0
1.0
1.0351190162670112e-230
1.0
7.276749268273074e-38
3.8189319202693326e-176
2.6979081497085705e-157
5.6084527364710576e-111
3.9535447334437505e-111
8.21344633204507